In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [4]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

# without symmetry
## loading data

In [108]:
'''
without symmetry eigenvalue and eigenvector projection 
'''
# strfile='2021-03-27-16_16_38_perturbationHomoZeroeta_data' # 500
# strfile='2021-03-27-17_02_42_perturbationHomoZeroeta_data' # 1000
# strfile='2021-03-27-17_58_42_perturbationHomoZeroeta_data' # 750
# strfile='2021-03-27-20_31_10_perturbationHomoZeroeta_data' # 750
# strfile='2021-03-27-20_44_33_perturbationHomoZeroeta_data' # 750

strfile='2021-03-27-20_01_56_perturbationHeterZeroeta_data' # 750
'''
previous data
'''
# strfile='2021-03-17-21_57_03_perturbationHomoZeroeta_data' # N=500
# strfile='2021-03-17-22_24_20_perturbationHomoZeroeta_data' #1000
# strfile='2021-03-17-20_35_21_perturbationHomoZeroeta_data' # 1000
# strfile='2021-03-17-22_46_39_perturbationHomoZeroeta_data'# 750
# '''# HETERO ZERO '''
# strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' 
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
ngavg=len(gaverageseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
Xprimebasexee=data['Xprimebasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
x0,y0=data['x0'],data['y0']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)

scale: 750.0
JEE: 1.0
eta: [0. 0. 0.] ; signs: [1. 1. 1.]


In [17]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

# '''
# calculate S0
# '''
# xticks = np.linspace(-0.08,0.08,2)
# xlims = [-0.08,0.08]
# yticks = np.linspace(-0.08,0.08,2)
# ylims = [-0.08,0.08]
# Z = np.zeros((NE*2-1,NE*2-1))
# for i in range(NE*2-1):
#     b1=eigvAm[i+1]
#     Z[i,i]=1/(-eigvAm[0])
# S0=np.zeros((NE*2,NE*2))
# R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
# S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

<ipython-input-17-cdfb9a45373f>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-17-cdfb9a45373f>:54: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)
<ipython-input-17-cdfb9a45373f>:56: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_the[:,i] = lprime_the*gsamples[i]
<ipython-input-17-cdfb9a45373f>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_the[:,i] = rprime_the*gsamples[i]
<ipython-input-17-cdfb9a45373f>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-17-cdfb9a45373f>:54: ComplexWarning: Casting complex values to real di

In [109]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
alphanum=1.0
# figE,axE=plt.subplots(figsize=(4,4))
# figL0,axL0=plt.subplots(2,1,figsize=(5,4))
# figR0,axR0=plt.subplots(2,1,figsize=(5,4))

## calculating the statistics of perturbation lambda
# Coding uses only one point [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:],axis=1)
axE.plot(gaverageseries,std_perturb_Lambda_num,color='darkgreen',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='green',linewidth=1.5,label='analytical',alpha=alphanum)
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.00,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
# std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
# std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2),dtype=complex),np.zeros((ngavg,2),dtype=complex)
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)


# >>>>>>>> numerical solution
axR0[0].plot(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
axR0[1].plot(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
# axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical',alpha=alphanum)
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical',alpha=alphanum)
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR0[0].scatter(gaverageseries,statesReig[:,i,0,1],s=5,c='darkred',marker='^',label='numerical',alpha=alphanum*0.5)
    axR0[1].scatter(gaverageseries,statesReig[:,i,1,1],s=5,c='darkblue',marker='^',label='numerical',alpha=alphanum*0.5)

ylims=[-0.00,0.020]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.020,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

# >>>>>>>> numerical solution
axL0[0].plot(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
axL0[1].plot(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
# axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical',alpha=alphanum)
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical',alpha=alphanum)
axL0[0].set_title(r'$\sigma_{l}^E$')
axL0[1].set_title(r'$\sigma_{l}^I$')
axL0[1].set_xticks(xticks)
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL0[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=5,c='darkred',marker='^',label='numerical',alpha=alphanum*0.5)
    axL0[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=5,c='darkblue',marker='^',label='numerical',alpha=alphanum*0.5)

ylims=[-0.00,0.045]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.045,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL0[1].set_xticks(xticks)





std delta_lambda numerical: 0.014197196965344543
std of perturbation right eigenvector, num: [0.00849479 0.00241897] ; theo: [0.00816497 0.0023094 ]
std of perturbation left eigenvector, num: [0.00882704 0.00372311] ; theo: [0.00847144 0.00356576]
<ipython-input-109-169649c8e34f>:60: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
<ipython-input-109-169649c8e34f>:61: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/

In [58]:
from scipy.special import comb, perm
C=comb(4,2)
print(C)

6.0


## eigenvalue spectrum

In [ ]:
'''
first, generate eigenvalues' spectrum (we didn't record the whole eigenvalues' spectrum)
'''

In [206]:
'''
without symmetry, 
eigvAm
'''
idxgavg,idxtrial=0,8
figtspt,axtspt=plt.subplots(figsize=(8,5))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=10,c='blue',alpha=0.5)
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0]),s=20,c='red',marker='^',alpha=0.5)
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#设置x坐标刻度数字或名称的位置
axtspt.xaxis.set_ticks_position('bottom')
#设置边框位置
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="darkblue", linewidth=2.5,linestyle='--',label=r'ellipse')
avgg=np.sum(np.sum(gmat))*2/4.0
avgg=np.sqrt(avgg)
xa = avgg*np.cos(theta)
ya = avgg*np.sin(theta)
axtspt.plot(xa, ya, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse')
print(avgg,gaverage_)



1.0 1.0 1.0 1.0
0.1
0.1 0.1
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [103]:
'''
loading patterns
'''
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.05,0.2,2)
# ylims = [-0.05,0.2]

## withoueigenvalue
xticks = np.linspace(-0.1,0.1,2)
xlims = [-0.1,0.1]
yticks = np.linspace(-0.02,0.08,2)
ylims = [-0.02,0.08]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
# pearsoncorr_rho=np.zeros((3,ngavg,neta,ntrial,2,nrank))
# thetamaxseries=np.zeros((3,ngavg,neta,ntrial,2,2))
# eigtotalseries=np.zeros((3,ngavg,neta,ntrial,nrank))
idxtrial=5#16#
idxgavg=4#9# 
cm='mc'
# fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:]),np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:])
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

## keep the same sign
if meanmE[0]<0:
    meanmE,meanmI,meannE,meannI=-meanmE,-meanmI,-meannE,-meannI
    mEvec,mIvec,nEvec,nIvec=-mEvec,-mIvec,-nEvec,-nIvec
''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=10.0,c=cm[1],alpha=0.25)
ax.scatter(nEvec,mEvec,s=10.0,c=cm[0],alpha=0.25)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=cm[1],linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=cm[0])
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=cm[1])
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(gaverageseries[idxgavg])

            


750
(2, 2)
(2, 2)
0.5
<ipython-input-103-321248a7bc9d>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-103-321248a7bc9d>:49: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-103-321248a7bc9d>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-103-321248a7bc9d>:64: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-103-321248a7bc9d>:75: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-103-321248a7bc9d>:76: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


# with symmetry
## loading data

In [133]:
strfile='2021-03-27-22_40_39_perturbationHomoeta_data' # 750
strfile='2021-03-27-23_15_34_perturbationHomoeta_data' # 500
data = np.load(strfile+'.npz')
etaseries=data['etaseries']
ngavg=len(gaverageseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)

neta=len(etaseries)

scale: 500.0
JEE: 1.0
eta: [1. 1. 1.] ; signs: [1. 1. 1.]


In [135]:
'''
determinant lemma, odd terms 
even order terms should be maitained
combination
'''
from scipy.special import comb, perm
gaverage
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    B2=C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2
    coeff2 = [1,-eigvAm[0],0,-B2]
    C=comb(4,2)
    B4=C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4
    # B4=(nAm.T@(Y_t@YT_t@Y_t@YT_t)@mAm)*fory**2*foryt**2*6
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

# fig,ax=plt.subplots(figsize=(5,3))  
# ax.plot(etaseries,lambda_num,'b',label='2 cut off')
# ax.set_aspect('equal')
# # ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# # ax.set_aspect('equal')


2.0
-2.5000000000000147 -2.5000000000000147
2.0
(-2.5278217585740665+0j) (-2.5282758619206427+0j)
2.0
(-2.554487863769397+0j) (-2.5561918545671856+0j)
2.0
(-2.580116174759492+0j) (-2.5837223922361567+0j)
2.0
(-2.60480612312529+0j) (-2.6108511293748218+0j)
2.0
(-2.628642487993115+0j) (-2.6375688292115105+0j)
2.0
(-2.6516982339592956+0j) (-2.6638716783570655+0j)
2.0
(-2.6740366810201457+0j) (-2.6897599638475325+0j)
2.0
(-2.6957131889132513+0j) (-2.7152370402541366+0j)
2.0
(-2.7167764822293496+0j) (-2.740308526549249+0j)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/lib/polynomial.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  p = p.astype(float)
<ipython-input-135-84306781a13d>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo2[idxeta]=lambda_test2[0]
<ipython-input-135-84306781a13d>:44: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo4[idxeta]=lam

In [134]:
'''
determinant lemma, odd terms 
even order terms should be maitained
original Algorithm, miss terms
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = 2
for idxeta,eta in enumerate(etaseries):
    etaf,etam,etal=eta*coeff[0],coeff[1]*eta,eta*coeff[2]
    bcoeffE0,bcoeffI0=(signeta[0]*etaf+signeta[1]*etam)*1/2.0,(signeta[2]*etal+signeta[1]*etam)*1/2.0
    print(bcoeffI0,bcoeffE0)
    B2=(bcoeffE0*JE-bcoeffI0*JI)*gaverage**2
    coeff2 = [1,-eigvAm[0],0,-B2]
    B4=(bcoeffE0**2*JE-bcoeffI0**2*JI)*gaverage**4
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    # B2=(bcoeffE0*y0E*x0E*NE)+(bcoeffI0*y0I*x0I*NE)
    # coeff2 = [1,-eigvJ0[0],0,-B2*eigvJ0[0]]
    # B4=(bcoeffE0**2*y0E*x0E*NE)+(bcoeffI0**2*y0I*x0I*NE)
    # coeff4 = [1,-eigvJ0[0],0,-B2*eigvJ0[0],0,-B4*eigvJ0[0]]#(eta*eigvAm[0]*gaverage])
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')
# ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# ax.set_aspect('equal')


0.0 0.0
(-2.5000000000000147+0j) (-2.5000000000000147+0j)
0.1111111111111111 0.1111111111111111
(-2.5278217585740674-5.551115123125783e-17j) (-2.528124566028627+1.942890293094024e-16j)
0.2222222222222222 0.2222222222222222
(-2.554487863769395+2.220446049250313e-16j) (-2.555624850653358-2.220446049250313e-16j)
0.3333333333333333 0.3333333333333333
(-2.580116174759492-1.6653345369377348e-16j) (-2.5825246849964594-4.440892098500626e-16j)
0.4444444444444444 0.4444444444444444
(-2.604806123125288+0j) (-2.608848172247852+3.3306690738754696e-16j)
0.5555555555555556 0.5555555555555556
(-2.6286424879931176+4.440892098500626e-16j) (-2.6346191862097688+2.220446049250313e-16j)
0.6666666666666666 0.6666666666666666
(-2.6516982339592983+0j) (-2.6598610392318833+3.3306690738754696e-16j)
0.7777777777777777 0.7777777777777777
(-2.674036681020144-4.440892098500626e-16j) (-2.684596269929125+1.1102230246251565e-16j)
0.8888888888888888 0.8888888888888888
(-2.6957131889132513+3.3306690738754696e-16j) (-2.70